In [ ]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import model_selection as cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [ ]:
ls data/

In [ ]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
df = pd.read_csv(data_path, low_memory = False)
df.head()

In [ ]:
no_dups = df.drop_duplicates()
no_dups.shape

In [ ]:
df.shape

In [ ]:
df[['bbl','zipcode','latitude','longitude']].to_csv("bbls_and_zips.csv", index = False)

In [ ]:
x = df[df['year_built'] != df['yearbuilt']]
x[x['year_built'] == 0.0].shape

In [ ]:
df[['gross_sqft_pluto','sale_price','price_per_sqft']].head()

In [ ]:
df = df[df['price_per_sqft'] != 0.0]
df.shape

In [ ]:
df = df[df['price_per_sqft'] >= 10]
df = df[df['price_per_sqft'] <= 5000]
df.shape

In [ ]:
count = collections.Counter(df['price_per_sqft'].astype(int))
count

In [ ]:
outliers = df.loc[df['price_per_sqft'].astype(int).isin([1,2,3,4,5,6912,7655,7949,10315,12089,14568])]
outliers[['latitude','longitude','price_per_sqft','sale_price','year_built','sale_date']].to_csv("outliers.csv")

In [ ]:
#output distribution of target variable to visualize in Tableau
import csv
x = df['price_per_sqft']
count = collections.Counter(x.astype(int))
with open("price_per_sqft_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Price Per Sqft', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [ ]:
#output distribution of sale price to visualize in Tableau
import csv
x = df['sale_price']
count = collections.Counter(x.astype(int))
with open("sale_price_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Sale Price', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [ ]:
def drop_cols(data, cols):
    return data.drop(cols, axis = 1)

In [ ]:
df = drop_cols(df, ['zonemap','sale_date','sale_price','year_built'])

In [ ]:
from sklearn import model_selection
def split_data(data):
    '''
    Splits data into training and test sets (0.8/0.2)
        Args: 
            data: Pandas dataframe
        Returns:
            data_train: Pandas dataframe used for training
            data_test: Pandas dataframe used for testing
    
    '''
    #Convert 'int64' into float; otherwise, sklearn throws a warning message
    columns = data.columns.values
    non_float = []
    for col in columns:
        if data[col].dtype != np.float64:
            non_float.append(col)
    for col in non_float:
        data[col] = data[col].astype(float)
    #drop NaN for crucial columns
    data= data.dropna(how = 'any', subset = ['price_per_sqft'])   
    #Split the data
    rs = model_selection.ShuffleSplit(train_size = 0.8, test_size=.2, random_state = 1, n_splits = 1)

    for train, test in rs.split(data):
        train_index = train
        test_index = test
    data_train = data.ix[train_index,:]
    data_test = data.ix[test_index,:]
    data_train.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    return data_train, data_test


In [ ]:
data_train, data_test = split_data(df)

In [ ]:
def fill_na(data_train, data_test):
    '''
    Fills NaN values with the mean of the column. Note we have already created dummy variables
    for columns with missing values.
    
    Args:
        data_train: Pandas dataframe used for training.
        data_test: Pandas dataframe used for testing.
    Returns:
        data_train: Pandas dataframe with no NaN values, ready for modeling.
        data_test: Pandas dataframe with no NaN values, ready for testing.
    
    '''
    data_train = data_train.apply(lambda x: x.fillna(x.mean()),axis=0)
    data_test = data_test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return data_train, data_test

In [ ]:
data_train, data_test = fill_na(data_train, data_test)

In [ ]:
print(data_train.shape, data_test.shape)


In [ ]:
cols = list(data_train.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('price_per_sqft')) #Remove b from list
data_train = data_train[cols+['price_per_sqft']]
data_test = data_test[cols+['price_per_sqft']]
data_train.columns.values

In [ ]:
data_train = data_train.drop(['public_recycling_bins_dist'], axis = 1)
data_test = data_test.drop(['public_recycling_bins_dist'], axis = 1)

In [ ]:
X_train = data_train.ix[:,:-1]
y_train = data_train.ix[:,-1]
X_test = data_test.ix[:,:-1]
y_test = data_test.ix[:,-1]
regr = linear_model.LinearRegression()
#regr.fit(X_train, y_train)
#mse = mean_squared_error(y_test, regr.predict(X_test))
#print('Mean_squared_error', mse)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_reg_final = RandomForestRegressor(n_estimators=100, n_jobs = -1)
RF_reg_final.fit(X_train, y_train)
print(mean_squared_error(y_test, RF_reg_final.predict(X_test)))

In [ ]:
bbls = []
file = open("data/Queens Light Rail BBL.csv", 'rb')
for line in file:
    bbls.append(line)
bbls = [float(i) for i in bbls]
bbls

In [ ]:
finance = pd.read_csv("finance_queens.csv")
finance.shape

In [ ]:
queens = pd.read_csv("data/nyc_pluto_16v1/QN.csv", low_memory = False)
queens.head()

In [ ]:
buildings_2 = pd.read_csv("pluto_finance_test.csv")
buildings_2.shape

In [ ]:
buildings_3 = pd.read_csv("data/merged/queens_2003_2016.csv")
buildings_3['price_per_sqft'].isnull().sum()

In [ ]:
buildings_3 = buildings_3.drop_duplicates()
buildings_3.shape

In [ ]:
df.bbl.isnull().sum()

In [ ]:
buildings_3.bbl.isnull().sum()

In [ ]:
buildings = pd.merge(queens, finance, how='right',
        left_on='BBL', right_on = 'bbl')
buildings.shape

In [ ]:
buildings_2 = buildings_2.drop_duplicates()
buildings_2.shape

In [ ]:
affected_properties = finance.loc[finance['bbl'].isin(bbls)]
affected_properties.shape

In [ ]:
list(X_test['bbl'])

In [ ]:
feature_importance =  RF_reg_final.feature_importances_
indices = np.argsort(feature_importance)[::-1][:27]

feature_dct = {}
# Print the feature ranking
print("Feature ranking:")

for f in range(20):
    feature_dct[X_test.columns.values[indices][f]] = feature_importance[indices[f]]
feature_dct

In [ ]:
from collections import OrderedDict
from operator import itemgetter

feature_dct = OrderedDict(sorted(feature_dct.items(), key=itemgetter(1), reverse = True))

In [ ]:
feature_dct.keys()

In [ ]:
data_train[['price_per_sqft'] + list(feature_dct.keys())].corr()

In [ ]:
neg_features = feature_dct.copy()
pos_features = feature_dct.copy()

#Use correlation matrix to determine which features are negatively correlated with our target variable
negs = ['gross_sqft_pluto','unitstotal']
for key in neg_features.keys():
    if key in negs:
        neg_features[key] = -neg_features[key]
    else:
        neg_features[key] = 0
for key in pos_features.keys():
    if key in negs:
        pos_features[key] = 0

In [ ]:
data_test['predicted'] = RF_reg_final.predict(X_test)
data_test['percent_difference'] = 100*(np.abs(data_test['predicted'] - data_test['price_per_sqft']).astype(float) / data_test['price_per_sqft'])
data_test[['yearbuilt','price_per_sqft','predicted','percent_difference']]

In [ ]:
acc = 100 * (data_test[data_test['percent_difference'] < 10.0].shape[0]/ data_test.shape[0])
acc

In [ ]:
results_df = pd.read_csv("preliminary_results.csv")
results_df

In [ ]:
import merge_pluto_finance_new as mpf

In [ ]:
queens_pluto = mpf.read_in_pluto(["queens"])

In [ ]:
queens_pluto.bbl.astype(int).isnull().sum()

In [ ]:
queens_finance = mpf.read_in_finance(["queens"], list(range(2003, 2017)))

In [ ]:
queens_finance.bbl.isnull().sum()

In [ ]:
print(queens_finance.shape)
print(queens_pluto.shape)

In [ ]:
print(queens_pluto.bbl.dtype)
print(queens_finance.bbl.dtype)

In [ ]:
queens_dtm = read_in_dtm(["queens"])

In [ ]:
import os
def read_in_dtm(boros, data_dir = 'data/dtm',
        filename = 'DTM_0316_Condo_Units.csv'):
    """
    Reads in the Digital Tax Map dataset and returns a dataframe with mapping
    from borough and condo number to unit BBL for the specified boroughs.

    Args:
        list(string) boros: list of all the boroughs to pull dtm data for
        string data_dir: a relative path as a string to folder containing the
            dtm data in csv format
        string filename: the name of the file containing the dtm condo unit data
    Returns:
        Pandas DataFrame
    """
    columns = ['CONDO_BORO', 'CONDO_NUMB', 'UNIT_BLOCK',
               'UNIT_LOT', 'UNIT_BBL', 'UNIT_DESIG']
    boro_names = ['manhattan', 'bronx', 'brooklyn', 'queens', 'statenisland']
    boro_codes = dict(zip(boro_names, range(1,6)))
    dtm = pd.read_csv(os.path.join(data_dir, filename), usecols=columns)
    dtm.columns = [col.strip().lower() for col in dtm.columns]
    dtm = dtm.dropna(subset = ['unit_bbl', 'condo_boro', 'condo_numb'])
    dtm.unit_bbl = dtm.unit_bbl.astype('int64')
    dtm = dtm.loc[dtm.condo_boro.isin(
        [boro_codes.get(boro) for boro in boros])]
    return dtm

In [ ]:
columns = ['CONDO_BORO', 'CONDO_NUMB', 'UNIT_BLOCK',
               'UNIT_LOT', 'UNIT_BBL', 'UNIT_DESIG']
data_dir = 'data/dtm'
filename = 'DTM_0316_Condo_Units.csv'
queens_dtm = pd.read_csv(os.path.join(data_dir, filename), usecols=columns)

In [ ]:
queens_dtm.isnull().sum()

In [ ]:
filepath = "data/open_nyc/subwaydist.csv"
subwaydist = pd.read_csv(filepath)
filepath = "data/open_nyc/some_dist_metrics.csv"
other_distances = pd.read_csv(filepath)

In [ ]:
len(pd.unique(other_distances.bbl)) == other_distances.shape[0]

In [ ]:
len(pd.unique(subwaydist.bbl)) == subwaydist.shape[0]

In [ ]:
finance_condo_updated = mpf.get_finance_condo_lot(pluto = queens_pluto,
        finance = queens_finance, dtm = queens_dtm)

In [ ]:
finance_condo_updated.bbl_pluto.isnull().sum()

In [ ]:
queens_pluto.bbl.isnull().sum()

In [ ]:
buildings = pd.merge(queens_pluto, finance_condo_updated, how='right',
        left_on='bbl', right_on = 'bbl_pluto',
        suffixes=['_pluto', '_finance'])

In [ ]:
buildings.shape

In [ ]:
buildings["price_per_sqft"] = buildings["sale_price"].astype('float64') / buildings["gross_sqft_pluto"]
buildings = buildings[ buildings["price_per_sqft"].notnull()]
buildings = buildings[ buildings["price_per_sqft"] > 0.]

In [ ]:
buildings.bbl_pluto 

In [ ]:
buildings.bbl

In [ ]:
queens_pluto.bbl

In [ ]:
finance_condos_only.bbl_finance

In [ ]:
queens_finance.columns

In [ ]:
queens_pluto.columns

In [ ]:
queens_dtm.columns

In [ ]:
finance_condo_updated.columns

In [ ]:
print(queens_finance.drop_duplicates().shape == queens_finance.shape)
print(queens_pluto.drop_duplicates().shape == queens_pluto.shape)
print(finance_condos_only.drop_duplicates().shape == finance_condos_only.shape)

In [ ]:
finance_condos_only.shape[0] + len(standard_bbls) 

In [ ]:
bbl_mappings.drop_duplicates().shape

In [ ]:
queens_finance.drop_duplicates(['bbl', 'sale_year']).shape == queens_finance.shape

In [ ]:
queens_finance.shape

In [ ]:
queens_finance.columns

In [ ]:
queens_finance["sale_year"] = [d.year for d in queens_finance.sale_date]

In [ ]:
print(finance_condo_updated.drop_duplicates().shape)
print(finance_condo_updated.shape)

In [ ]:
finance_condo_updated.columns

In [ ]:
bbl_mappings.bbl_finance.drop_duplicates().shape

In [ ]:
dtm_cols_to_keep = ['unit_bbl', 'condo_boro', 'condo_numb']
pluto_cols_to_keep = ['bbl', 'block', 'borocode', 'condono']

finance_condos_only = pd.merge(queens_finance, queens_dtm[dtm_cols_to_keep],
    how='inner', left_on=['bbl'], right_on=['unit_bbl'])

# for condos: finance.bbl == finance_condos_only.unit_bbl
#             finance_condos_only.bbl_pluto == pluto.bbl
finance_condos_only = pd.merge(queens_pluto[pluto_cols_to_keep],
    finance_condos_only, how='inner',
    left_on=['borocode', 'block', 'condono'],
    right_on=['condo_boro', 'block', 'condo_numb'],
    suffixes=['_pluto', '_finance'])

finance_condos_only = finance_condos_only[
        ['bbl_pluto', 'bbl_finance']].drop_duplicates()
         # duplicates only if a bbl is listed in multiple years
finance_condos_only = finance_condos_only.loc[lambda df:
        np.floor(df.bbl_pluto / 1e4) == np.floor(df.bbl_finance / 1e4)]

# get a list of bbls that are not condos (same in pluto and finance)
standard_bbls = list(set(queens_finance.bbl).difference(
                    set(finance_condos_only.bbl_finance)))
# combine condo bbls that differ with standard bbls that are the same
bbl_mappings = finance_condos_only.append(pd.DataFrame.from_dict(
    {'bbl_pluto': standard_bbls, 'bbl_finance': standard_bbls}
))
bbl_mappings = bbl_mappings.reset_index(drop = True)

finance_condo_updated = pd.merge(queens_finance, bbl_mappings,
    how='left', left_on='bbl', right_on='bbl_finance')


In [ ]:
# finance condo updated: remove bbl/block/bbl_finance
# retain only bbl_pluto to match with pluto.bbl in merge
finance_condo_updated = finance_condo_updated.drop(
    ['block', 'bbl', 'bbl_finance'], axis=1)

# Remove duplicate bbls by returning only the most recent sales data
# for each BBL and year
finance_condo_updated = finance_condo_updated.reset_index()
finance_condo_updated["sale_year"] = [d.year for d in
    finance_condo_updated.sale_date]
grouped = finance_condo_updated.groupby(['bbl_pluto', 'sale_year'])
max_idx_by_bbl = grouped['sale_price'].idxmax().values
finance_condo_updated = finance_condo_updated.loc[max_idx_by_bbl]

In [ ]:
queens_finance_raw = pd.DataFrame()
data_dir = "data/finance_sales"
years = list(range(2003, 2017))
boros = ['queens']

for year in years:
        for borough in boros:
            print("Pulling Finance data for {}_{}".format(year, borough))
            boro_year = mpf.read_in_boro_year_data(borough, year, data_dir)
            queens_finance_raw = queens_finance_raw.append(boro_year)

In [ ]:
print(finance_condos_only.shape)
print(finance_condos_only.drop_duplicates().shape)
print(finance_condos_only.drop_duplicates('bbl_finance').shape)

In [ ]:
np.where(bbl_mappings.bbl_finance.duplicated(keep=False))

In [ ]:
print(finance_condo_updated.shape)
print(finance_condo_updated.drop_duplicates().shape)
print(finance_condo_updated.drop_duplicates(['bbl_pluto', 'sale_year']).shape)

In [ ]:
finance_condo_updated.columns

In [ ]:
queens_pluto.columns

In [ ]:
buildings = pd.merge(queens_pluto, finance_condo_updated, how='right',
        left_on='bbl', right_on = 'bbl_pluto',
        suffixes=['_pluto', '_finance'])

In [ ]:
buildings.shape

In [ ]:
buildings.shape

In [ ]:
buildings["price_per_sqft"] = buildings.sale_price.astype('float64') / buildings.gross_sqft_pluto
buildings = buildings.loc[ buildings.price_per_sqft.notnull()]
buildings = buildings.loc[ buildings.price_per_sqft > 0.]

In [ ]:
all(buildings.bbl == buildings.bbl_pluto)

In [ ]:
buildings.columns

In [127]:
import merge_pluto_finance_new as mpf
from importlib import reload
reload(mpf)

<module 'merge_pluto_finance_new' from '/Users/jacqueline/Desktop/reveal-estate/merge_pluto_finance_new.py'>

In [128]:
boros = ['all']
years = ['all']
if years == ["all"]:
    years = list(range(2003, 2017))
if boros == ["all"]:
    boros = ["manhattan", "brooklyn", "queens", "bronx", "statenisland"]

In [129]:
# Convert to lowercase and remove spaces in borough names
boros = ["".join(boro.lower().split()) for boro in boros]

In [ ]:
print("Getting PLUTO data for: {}".format(boros))
pluto = mpf.read_in_pluto(boros)
print("Getting Finance data for: {} and {}".format(boros, years))
finance = mpf.read_in_finance(boros, years)
print("Getting DTM Condo Unit data for: {}".format(boros))
dtm = mpf.read_in_dtm(boros)

Getting PLUTO data for: ['manhattan', 'brooklyn', 'queens', 'bronx', 'statenisland']


In [11]:
pluto = mpf.read_in_pluto(boros)

In [8]:
dtm.dtypes

condo_boro    float64
condo_numb      int64
unit_block      int64
unit_lot        int64
unit_bbl        int64
unit_desig     object
dtype: object

In [20]:
pluto.dtypes

borough              object
block                 int64
schooldist          float64
council             float64
zipcode             float64
ltdheight            object
splitzone            object
bldgclass            object
landuse             float64
easements             int64
ownertype            object
comarea             float64
resarea             float64
numbldgs            float64
numfloors           float64
unitsres            float64
unitstotal          float64
lotfront            float64
lotdepth            float64
bldgfront           float64
bldgdepth           float64
proxcode            float64
irrlotcode           object
lottype             float64
bsmtcode              int64
yearbuilt           float64
builtcode            object
histdist              int64
landmark              int64
borocode              int64
bbl                   int64
condono               int64
xcoord              float64
ycoord              float64
zonemap              object
latitude            

In [22]:
import pandas as pd

In [37]:
pluto_raw_queens.splitzone.value_counts(dropna=False)

N      314365
Y       10039
NaN       152
Name: splitzone, dtype: int64

In [38]:
pluto_raw_queens.irrlotcode.value_counts(dropna=False)

N      275303
Y       49101
NaN       152
Name: irrlotcode, dtype: int64

In [39]:
pluto_raw_manhattan.ltdheight.value_counts(dropna=False)

NaN      42286
LH-1A      812
LH-1        60
Name: ltdheight, dtype: int64